In [4]:
import torch
from transformers import pipeline

# 1. Summarization pipeline (multilingual models like 'facebook/mbart-large-50-many-to-many-mmt' for real use)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# 2. Action Item Extraction (using zero-shot classification as a proxy)
action_item_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def summarize_transcript(transcript):
    summary = summarizer(transcript, max_length=130, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def extract_action_items(transcript):
    # Split transcript into sentences (simple split; for production use NLP sentence segmentation)
    sentences = transcript.split('.')
    action_labels = ["action item", "decision", "follow up", "task", "to do"]
    action_items = []
    for sentence in sentences:
        if sentence.strip():
            result = action_item_classifier(sentence, action_labels)
            # If 'action item' is the top label and score > 0.5, treat as an action item
            if result['labels'][0] == "action item" and result['scores'][0] > 0.5:
                action_items.append(sentence.strip())
    return action_items

# --- Example Usage ---
if __name__ == "__main__":
    transcript = """
## Meeting Summary

The meeting commenced with a review of last quarter’s sales figures, highlighting a 10% growth despite seasonal fluctuations. John presented the updated financial spreadsheet, noting that marketing expenses increased by 15%, primarily due to the new campaign for the product launch. Sarah asked about the ROI of these campaigns, and John agreed to provide a detailed report at the next meeting.

The team then discussed customer feedback from the recent survey. Most respondents appreciated the improvements in the mobile app, but several raised concerns about data privacy and login issues. Priya suggested implementing a two-factor authentication feature, and the engineering team agreed to evaluate its feasibility.

The product roadmap for Q4 was next on the agenda. Alice proposed prioritizing the mobile app update, focusing on new features such as offline access and enhanced notifications. Bob recommended a phased rollout to minimize risks, and everyone concurred. Tim volunteered to draft a timeline for the update and share it with the group by Friday.

Marketing strategies for the upcoming quarter were discussed. Carol presented options for expanding social media outreach, including targeted ads and influencer partnerships. Bob cautioned that the team should monitor engagement metrics closely. Priya will coordinate with Carol to develop sample ad copy by next week.

The group considered international expansion opportunities, especially in the Latin American market. Sarah explained the regulatory requirements for localization and translation. Alice agreed to start researching local vendors for translation services, and John will oversee the legal compliance process.

The team also reviewed project management practices. Tim suggested adopting a new task-tracking tool to improve transparency and accountability. Bob will evaluate available platforms and present his findings at the next meeting.

Carol announced a client meeting scheduled for next Wednesday, focusing on the new app features and customer feedback. Priya will prepare a presentation, and Bob will provide the latest user engagement statistics.

The meeting ended with a review of action items and deadlines. Alice reminded everyone to update their tasks in the shared project tracker, and Sarah confirmed the next meeting will be held on Thursday at 2 PM.

---


    
    """
    
    print("Meeting Summary:")
    print(summarize_transcript(transcript))
    
    print("\nAction Items:")
    for item in extract_action_items(transcript):
        print("- " + item)
        

Device set to use cpu
Device set to use cpu


Meeting Summary:
The team discussed customer feedback from the recent survey. Most respondents appreciated the improvements in the mobile app, but several raised concerns about data privacy and login issues. Priya suggested implementing a two-factor authentication feature, and the engineering team agreed to evaluate its feasibility.

Action Items:
- Alice agreed to start researching local vendors for translation services, and John will oversee the legal compliance process
- Bob will evaluate available platforms and present his findings at the next meeting
- - Bob to evaluate new task-tracking platforms and present findings at next meeting
- - Sarah to schedule the next team meeting for Thursday at 2 PM
